# Advent of Code 2022: Day 19
https://adventofcode.com/2022/day/19


## Part 1
Get the maximum number of geodes in 24 minutes for each of a number of 'blueprints' with information on how much different resource-collecting robots cost to produce.

### Install the mip module

In [1]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Get the data into a list of strings

In [2]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Function to preprocess the data

In [3]:
def preprocess_data(data):
  
  # Setup output list
  output_list = []
  
  # Go through each blueprint
  # in the data.
  for item in data:
    
    # Setup temporary list to hold 
    # the costs for a blueprint
    temp_list = []
    
    # Go through each robot in a blueprint
    item_split = item.lstrip('Blueprint ').split('.')[:-1]
    for inner_item in item_split:
      
      # Setup temporary list to hold
      # the cost for a single robot.
      inner_temp_list = []
      
      # Go through each word in the 
      # description of a robot.
      inner_split = inner_item[1:].split(' ')
      for c in inner_split:
        
        # If the word is a digit, convert it
        # to a number and append it to the list.
        if c.isdigit():
          inner_temp_list.append(int(c))
      
      # Append the cost for a robot
      # to the blueprint list.
      temp_list.append(inner_temp_list)
    
    # Append the blueprint list
    # to the output list.
    output_list.append(temp_list)
  return output_list

### Function to calculate the maximum number of geodes for every blueprint

In [4]:
# Import required classes and functions
from mip import Model, xsum, maximize, BINARY

def sum_quality_levels(data, minutes, part2 = False):

  # Preprocess the data and setup
  # the sum.
  blueprints = preprocess_data(data)
  quality_sum = 0
  
  # Get a range object for 
  # the number of minutes.
  I = range(minutes)
  
  # For part 2, only get the first three
  # blueprints, and set the counter to 1.
  if part2 == True:
    blueprints = blueprints[:3]
    quality_sum = 1
  
  # Go through every blueprint
  for b in range(len(blueprints)):
    
    # Get the blueprint and the
    # cost for every robot.
    blueprint = blueprints[b]
    r1_cost = blueprint[0]
    r2_cost = blueprint[1]
    r3_cost = blueprint[2]
    r4_cost = blueprint[3]

    # Initialize the optimizer model
    m = Model("geode")

    # Add the variables to optimize to the model. In this case it is
    # a 2d array where a row corresponds to the robot type and the
    # columns corresponds to the current minute. The values will be
    # 1 if a robot of a type was produced in the current minute, and
    # 0 otherwise. 
    x = [[m.add_var(var_type=BINARY) for i in I] for j in range(4)]

    # The objective is to optimize the current function, which
    # says that every geode robot (robots of type 4) produce 1
    # geode for every minute they are active, except the minute 
    # they were produced. 
    m.objective = maximize(xsum((minutes-1-i) * x[3][i] for i in I))

    # Add a constraint to the model, which says that in the first
    # minute, we only have one robot of type 1.
    m += x[0][0] == 1

    # Add a constraint to the model for every minute, which 
    # says that for a given minute, only a single robot 
    # of a single type can be produced. 
    for i in I:
      m += (x[0][i]+x[1][i]+x[2][i]+x[3][i]) <=1

    # Add three constraints to the model for every minute, except
    # the first and last minutes, that deal with the costs of the robots.
    for i in range(1,minutes-1):
      
      # Every robot requires ore to produce, meaning that for a given
      # minute, the combined ore cost of every active robot cannot be
      # greater than the amount of ore that has so far been produced. 
      m += -xsum(x[0][j]*r1_cost[0] for j in range(1,i+1)) - xsum(x[1][j]*r2_cost[0] for j in range(i+1)) - xsum(x[2][j]*r3_cost[0] for j in range(i+1)) - xsum(x[3][j]*r4_cost[0] for j in range(i+1)) + xsum((i-j-1)*x[0][j] for j in range(i))+1 >= 0
      
      # The robot of type 3 costs clay to produce, meaning that for a 
      # given minute, the combined clay cost of every active robot of
      # type 3 cannot be greater than the amount of clay that has so
      # far been produced.
      m += -xsum(x[2][j]*r3_cost[1] for j in range(i+1)) + xsum((i-j-1)*x[1][j] for j in range(i)) >= 0
      
      # The robot of type 4 costs obsidian to produce, meaning that for 
      # a given minute, the combined obsidian cost of every active robot 
      # of type 4 cannot be greater than the amount of obsidian that has 
      # so far been produced.
      m += -xsum(x[3][j]*r4_cost[1] for j in range(i+1)) + xsum((i-j-1)*x[2][j] for j in range(i)) >= 0

    # Optimize the objective
    m.optimize()
    
    # Get the number of geodes
    # produces for the blueprint.
    quality = sum((minutes-1-i) * x[3][i].x for i in I)
    
    # Add to or multiply the sum based
    # on the part. 
    if part2 == True:
      quality_sum = quality_sum * quality
    else:
      quality_sum += (b+1)*quality
  return int(quality_sum)

In [5]:
sum_quality_levels(data_list, 24)

1427

## Part 2
Get the maximum number of geodes in 32 minutes, for just the first 3 blueprints.

In [6]:
sum_quality_levels(data_list, 32, True)

4400